# Data from GTEx
Link: https://www.gtexportal.org/home/downloads/adult-gtex/bulk_tissue_expression


**Bulk-Tissue-Expression:**

Contains the gene expression in *healthy* tissue samples from adults. For each gene, the transcription value for each tissue is listed.

**Output file format:**
* Ensemble ID
* TPM value

## Load Data and melt down to long format
→ takes 20 minutes with 3000 rows per chunk

The format of the input file is a column per tissue.
We load the data in chunks and melt it down to a long format with columns for the gene name, tissue and TPM value.


In [ ]:
import pandas as pd
import dataframe_image as dfi
import gzip

import gc
import time
import os
import urllib.request

In [ ]:
# Function to analyze the dataset
def dataset_analysis(df):
    global missing_values, min_tpm, max_tpm
    
    missing_values =+ df.isnull().sum()
     
    if df['tpm'].min() < min_tpm:
        min_tpm = df['tpm'].min()
        
    if df['tpm'].max() > max_tpm:
        max_tpm = df['tpm'].max()

In [ ]:
# download the data
url = "https://storage.googleapis.com/adult-gtex/bulk-gex/v8/rna-seq/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz"

zip_file_name = "../import_data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz"

file_name = "../import_data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct"

os.makedirs("../import_data", exist_ok=True)
os.makedirs("../import_data/GTEx", exist_ok=True)


if not os.path.exists(zip_file_name):
    urllib.request.urlretrieve(url, zip_file_name)

if not os.path.exists(file_name):
    with gzip.open(zip_file_name, 'rb') as f_in:
        with open(file_name, 'wb') as f_out:
            f_out.write(f_in.read())

In [ ]:
output_file = "../processed_data/GTEX_healthy.csv"

os.makedirs("../processed_data", exist_ok=True)

# Initialize an output file and overwrite if already exists
with open(output_file, 'w') as f_out:
    f_out.write("id,tissue,tpm\n")

missing_values = 0
min_tpm = float('+inf')
max_tpm = float('-inf')

chunksize = 2000
save_threshold = 10000

output_chunks = []
outputrows = 0

with pd.read_csv(file_name, delimiter="\t", skiprows=2, chunksize=chunksize) as reader:
    counter = 0

    for df_chunk in reader:
        start_time = time.time()
        df_list = []
        
        # cleanup
        df_chunk = df_chunk.drop(columns=['Description'])
        df_chunk.rename(columns={'Name':'id'}, inplace=True)
        df_chunk['id'] = df_chunk['id'].str.split('.').str[0]
        
        df_long = pd.melt(df_chunk, id_vars=['id'], var_name='tissue', value_name='tpm', ignore_index=True)


        df_long["tpm"] = df_long["tpm"].astype(float)
        
        output_chunks.append(df_long)
        outputrows += df_long.shape[0]
        
        dataset_analysis(df_long)
        
        if outputrows >= save_threshold:
            pd.concat(output_chunks).to_csv(output_file, mode='a', header=False, index=False)
            output_chunks.clear()
        
        # Clear memory
        del df_long
        
        end_time = time.time()
        duration = end_time - start_time        
        print(f"{counter} / 56200 processed in {duration:.2f} seconds")
        
        counter += chunksize
        
if output_chunks:
    pd.concat(output_chunks).to_csv(output_file, mode='a', header=False, index=False)
        
print(f"Output file contains {outputrows} rows")

### Analyze the dataset

In [ ]:
print(f"Missing values:\n"
      f"{missing_values}\n")
print(f"Min TPM: {min_tpm}")
print(f"Max TPM: {max_tpm}")

## Save grouped data

In [ ]:
file = "../processed_data/GTEX_healthy.csv"
output_file = "../processed_data/GTEX_healthy_temp.csv"

#chunksize = 200000000
chunksize = 20000000

# Initialize output file and overwrite if already exists
with open(output_file, 'w') as f_out:
    f_out.write("id,tmp sum,tmp count\n")
    
with (pd.read_csv(file, chunksize=chunksize) as reader):
    for df_chunk in reader:
        start_time = time.time()
        df_mean = df_chunk.drop(columns=["tissue"])
        df_mean = df_mean.groupby('id').agg(['sum','count'])
        df_mean.to_csv(output_file, mode='a', header=False, index=True)

        end_time = time.time()
        duration = end_time - start_time
        print(f"{df_mean.shape[0]} rows processed in {duration:.2f} seconds")

        del df_mean
        gc.collect()

## Load temp file to calculate mean TPM

In [ ]:
df_mean = pd.read_csv("../processed_data/GTEX_healthy_temp.csv")
df_mean = df_mean.groupby('id').sum()
df_mean['tpm'] = df_mean['tmp sum']/df_mean['tmp count']
df_mean

## Clean up

In [ ]:
df_mean = df_mean.drop(columns=['tmp sum', 'tmp count'])

df_mean.reset_index(inplace=True)

df_mean.rename(columns={'tpm':'healthy TPM',
                        'id': 'Gene ID'}, inplace=True)

# reorder columns
df_mean = df_mean[['Gene ID', 'healthy TPM']]

In [ ]:
df_mean

## Save final file

In [ ]:
df_mean.to_csv("../processed_data/GTEX_healthy_mean.csv")
print(f'There are {df_mean.shape[0]} genes in the final file')

In [ ]:
dfi.export(df_mean.head(5), "../tex/figures/03_01_GTEX_healthy_mean.png")